In [7]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [8]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [9]:
scenarios = ["Add New Shipping Address with Location's Address Incorrectly Filled"]

In [10]:
prompt_uc="""Checkout
Use Case ID: UC-002
Description: This feature allows users to complete the purchase of items in their shopping cart.
Actor: User

Preconditions: 
User is logged into their Shopee account.
Postconditions
The order is successfully placed.
The user receives an order confirmation.
Basic Flow
Step 1: User perform feature 'Purchase'.
Step 2: System displays the order summary, including items, quantities, prices, and total cost.
Step 3: System set shipping address as default address. User can change or add new a shipping address.
Step 4: System set default payment method as COD. There are other payment method such as: Shopee Pay, Google Pay, credit. User can change a payment method by select new payment method.
Step 5: User can add application voucher or shop voucher to order by select available voucher or enter voucher code.
Step 6: System set default shipping option as Standard Express. User can change shipping option.
Step 7: User can add note for shop.
Step 8: System update the total cost based on user changes. User reviews the order details.
Step 9: User clicks on the "Place Order" button.
Step 10: System processes the payment and confirms the order.
Step 11: System displays the order confirmation and details.
Alternative Flows:
1.If user has no default shipping address, system will force user to add a new shipping address.
2. If user want to change shipping address: 
-user click button change shipping address. system will display a list of saved address and user can select one.
3. If user want to add a new shipping address: 
-user click button change shipping address, add new address
-user fill the form: 
        + Full name: contains only letter and cannot be less than 2 letters
        + phone number: must be in format (+84) xxx xxx xxx with all the x is number
        + city: dropdown menu list cities provided by the platform
        + district: dropdown menu list districts provided by the platform
        + ward: dropdown menu list wards provided by the platform
        + location's address: must be defined in the map
        + Address type (optional, ratio): own house, working place
- user click button 'Finish' to add shipping address. 
4. if user enter a voucher code (for application voucher/ shop voucher):  
- user input a correct voucher code.
- user click 'apply' button.
- system add voucher to order and discount the order based on the voucher input.
5. If user change shipping option: 
- user click on shipping option change button.
- system display a list of available shipping options.
- user select one.

Exception Flow
1. user cannot add disable voucher.
2. user cannot apply an incorrect voucher code.
3. If the input is not match the require condition or the inputted location's address cannot be found in the map, system send notification and prevent user from adding new shipping address.
4. If there is an issue with the order:
System displays an error message and suggests corrective actions (e.g., updating the shipping address, checking payment details)."""

In [11]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form.
"""

In [12]:
for scenario in scenarios:
    i = 1
    while(i<=1):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            print(gpt_response)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")

```json
{
  "Test Case 1": {
    "testCaseName": "Add New Shipping Address with Correct Details",
    "objective": "Verify that the user can successfully add a new shipping address with all details correctly filled",
    "testSteps": [
      "Step 1: Log into the Shopee account.",
      "Step 2: Navigate to the checkout page.",
      "Step 3: Click on the 'Change Shipping Address' button.",
      "Step 4: Click on 'Add New Address'.",
      "Step 5: Fill in the form with correct details including a valid location's address.",
      "Step 6: Click on the 'Finish' button."
    ],
    "expectedResult": "The new shipping address is successfully added and selected for the order.",
    "explanation": "This test case ensures that users can add a new shipping address by following the correct steps and filling in the required information accurately. It is related to the basic and alternative flow where a user needs to add a new shipping address."
  },
  "Test Case 2": {
    "testCaseName": "Add

KeyboardInterrupt: 